In This Notebook, we are building an AI Agent which is capable of recommeding you a real-estate based on your preferences.

In [ ]:
# !pip install -r requirements.txt

In [13]:
import os
from utils import PropertyCollection
from config import PropertyQuestionnaireConfig
from langchain.output_parsers import PydanticOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [2]:
os.environ["OPENAI_API_KEY"] = "voc-3536329941266772997668678e47bca8fda1.32828252"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

In [4]:
# Load the chatbot config
chat_bot_config =PropertyQuestionnaireConfig()

# PropertyCollection is implemented in utils
parser = PydanticOutputParser(pydantic_object=PropertyCollection)

In [ ]:
# Prompt template to generate real estate listings
prompt = PromptTemplate(
    template="{instruction}\n{sample}\n{format_instructions}\n",
    input_variables=["instruction", "sample"],
    partial_variables={"format_instructions": parser.get_format_instructions},
)

# Generate the prompt
query = prompt.format(
    instruction=chat_bot_config.instruction,
    sample=chat_bot_config.sample_listing,
)

In [ ]:
llm = ChatOpenAI(model=chat_bot_config.chat_bot_model temperature=0)
# Invoke the model to generate real estate listings
response = llm.invoke(query)

In [ ]:
from fastapi.encoders import jsonable_encoder
import pandas as pd

# Parse the response and convert it to a DataFrame
parsed_result = parser.parse(response.content)
df = pd.DataFrame(jsonable_encoder(parsed_result.properties))
df.head()

,location,price,num_bedrooms,num_bathrooms,square_footage,details,area_overview
0,Green Oaks,800000,3,2,2000,Welcome to this eco-friendly oasis nestled in ...,"Green Oaks is a close-knit, environmentally-co..."
1,Downtown Loft District,1200000,2,2,1800,Luxury loft living in the heart of the city! T...,The Downtown Loft District is a vibrant neighb...
2,Suburban Meadows,600000,4,3,2500,"Welcome home to this spacious 4-bedroom, 3-bat...",Suburban Meadows is a family-friendly communit...
3,Beachfront Paradise,1500000,5,4,3500,Live the beachfront dream in this stunning 5-b...,Beachfront Paradise offers a relaxed coastal v...
4,Mountain Retreat,900000,3,2,2200,Escape to this peaceful mountain retreat nestl...,The Mountain Retreat neighborhood is a nature ...


In [ ]:
# Save the DataFrame to a CSV file
df.to_csv('Real_Estate_Listings.csv', index_label = 'id')

In [5]:
# Load the real estate listing from Excel file

loader = CSVLoader(file_path='./Real_Estate_Listings.csv')
real_estate_listings = loader.load()

In [6]:
embeddings = OpenAIEmbeddings()

# Split the real estate listings into chunks for faster processing
splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = splitter.split_documents(real_estate_listings)

# Create a vector store from the real estate listings with embedding function
db = Chroma.from_documents(split_docs, embeddings)

/tmp/ipykernel_6242/1271620779.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [7]:
# Example of a search query for similarity search to get the best match
search_query = "A comfortable three-bedroom house with a spacious kitchen and a cozy living room."
matched_real_estate = db.similarity_search(search_query, k=1)

In [8]:
# Format questions and answers into a readable format
formatted_qa = "\n".join([f"{q} {a}" for q, a in zip(chat_bot_config.questions, chat_bot_config.answers)])

# Format best real estate matches
formatted_listings = "\n".join([f"{i+1}. {doc.page_content}" for i, doc in enumerate(matched_real_estate)])

# Define the recommender prompt (without memory)
RECOMMENDER_TEMPLATE = """
Introduce yourself as AI Real Estate Agent and provide a tailored response to the buyer's preferences. withe following instructions:
Based on the following answers, I recommend the following real estate listings:

{formatted_qa}

------------
Here are the top matching listings:

{formatted_listings}

------------
Formulate a reply that not only addresses the questions above, but also delivers an engaging, unique, and tailored explanation that resonates with the given preferences. This requires subtly highlighting key property features that closely match the buyer's desires.
"""



In [9]:
# Define the recommender prompt
llm = ChatOpenAI(model_name=chat_bot_config.chat_bot_model, temperature=chat_bot_config.chat_bot_temp, max_tokens=chat_bot_config.chat_bot_max_tokens)

/tmp/ipykernel_6242/2443102680.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name=chat_bot_config.chat_bot_model, temperature=chat_bot_config.chat_bot_temp, max_tokens=chat_bot_config.chat_bot_max_tokens)


In [ ]:
# Create the PromptTemplate
PROMPT = PromptTemplate(
    input_variables=["formatted_qa", "formatted_listings"],
    template=RECOMMENDER_TEMPLATE
)

# Initialize the LLMChain with the prompt
chain = LLMChain(llm=llm, prompt=PROMPT)

/tmp/ipykernel_6242/2582858599.py:11: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=PROMPT)


In [11]:
# Get the response from the chatbot
response = chain.run({"formatted_qa": formatted_qa, "formatted_listings": formatted_listings})

/tmp/ipykernel_6242/217343661.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run({"formatted_qa": formatted_qa, "formatted_listings": formatted_listings})


In [12]:
print(response)

Hello there! I'm your AI Real Estate Agent, here to help you find your dream home. Based on your preferences, I have the perfect listing for you:

Listing ID: 16 - Modern Townhouse
Price: $800,000
Bedrooms: 3
Bathrooms: 2
Square Footage: 2000

This modern townhouse offers a comfortable three-bedroom layout with a spacious kitchen, perfect for cooking and entertaining. The cozy living room is ideal for relaxing after a long day. Located in a trendy urban neighborhood, you'll have access to restaurants and theaters while still enjoying suburban tranquility.

In terms of your top priorities, this property is situated in a quiet neighborhood with good local schools nearby. You'll also have convenient shopping options just steps away. The backyard is perfect for gardening, and the two-car garage provides ample storage space. Plus, the modern, energy-efficient heating system will keep you comfortable year-round.

For transportation, you'll have easy access to a reliable bus line, proximity t